<h1 style="text-align: justify;"><strong>Capstone Final Project Report</strong></h1>
<h4 style="text-align: justify;">Opening a Latino Restaurant in Toronto</h4>
<p style="text-align: justify;">In this final project of the capstone of the IBM Data Science Professional Program, I will define a business problem and try to solve it with the tools presented in the previously mentioned program.</p>
<p style="text-align: justify;">I will explain the process step-by-step, and also provide all the necessary background.</p>
<p style="text-align: justify;"><strong>Introduction/Business Problem</strong></p>
<p style="text-align: justify;"><span class="rc-PlainTextView">Toronto is the most populous city in Canada and is home to a variety of nationalities. People from other parts love to have their country food, and also locals can enjoy different tastes. </span></p>
<p style="text-align: justify;"><span class="rc-PlainTextView">This project will be about figuring out if opening a Latino Restaurant is a good idea, and if it is, where to open it. We will analyze different neighborhoods, the demographics and the current offer to better match-up our proposal. The stakeholders of this project will be: </span></p>
<ul>
<li style="text-align: justify;"><span class="rc-PlainTextView"><em>Businessman/Chefs interested in opening/investing restaurants.</em> The project will be useful to them to identify key areas for the business.</span></li>
<li style="text-align: justify;"><span class="rc-PlainTextView"><em>Latino community</em>, who will be grateful to identify areas where they can find food that matches their taste. </span></li>
<li style="text-align: justify;"><span class="rc-PlainTextView"><em>City guides</em>, they will gain insight into the different attributes areas have and how they can improve their service to match the demands in different areas.</span></li>
</ul>
<p><span class="rc-PlainTextView"><strong>Data</strong></span></p>
<p><span class="rc-PlainTextView">To achieve our objective, we will use different datasets: </span></p>
<ul>
<li><span class="rc-PlainTextView"><em>List of Postal Code of Canada</em> (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M): This is a dataset that lists the different neighborhoods in Toronto, having the Postcode, Borough and the Neighbourhood as attributes. For example, MP4 with Borough York and Neighbourhood Weston. </span></li>
<li><span class="rc-PlainTextView"><em>Toronto Geospatial Data</em> (https://cocl.us/Geospatial_data): This dataset list all the postal codes with latitude and longitude as attributes. For example, MP4 with Longitude -79 and latitude 32. </span></li>
<li><span class="rc-PlainTextView"><em>Demographics of Toronto</em> (https://en.wikipedia.org/wiki/Demographics_of_Toronto): We will use this Wikipedia page to have insight into where do Latinos live. The neighborhood Latino racial density.</span></li>
<li><span class="rc-PlainTextView"><em>Bicycle Parking Racks Data</em>Toronto Open Data: We will use this database to know how many bicycle parking racks are in the Postalcode because of the food delivery trends.</span></li>
<li><span class="rc-PlainTextView"><em>Foursquare API</em>: We will use this API to extract more information about neighborhoods like restaurants (current offer). For example, the names of Latino restaurants with their latitude and longitude. </span></li>
    </ul>

<p><strong>Methodology</strong></p>

<h4><i>Web-scraping Postal Codes Data</i></h4>
<p> First step is to import the table from Wikipedia. I will use BeautifulSoup to extract the HTML text from the Wikipedia page and then convert it to a df. We will be scraping the List of Postal Code of Canada from Wikipedia</p>

In [1]:
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"lxml")
#print(soup.prettify())

<P>Once we have the HTML Text, we have to find the table we are searching for (Postcodes):</P>

In [2]:
My_table = soup.find("table",{"class":"wikitable sortable"})
#My_table

With the HTML Text of the table, now we can read it and translate to a DF.

In [3]:
import pandas as pd
dfs = pd.read_html(str(My_table))
#dfs
df = dfs[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


<p> I will remove the "not assigned" Borough... </p>

In [4]:
df = df[df.Borough != "Not assigned"]
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


<p> Since there are not "not assigned" Neighborhood, I didn't replace them with the Borough </p>

In [5]:
df[df.Neighbourhood == "Not assigned"]

,Postcode,Borough,Neighbourhood


In [6]:
index = df.Postcode
index.value_counts()

M9V    8
M8Y    8
M5V    7
M4V    5
M8Z    5
      ..
M9P    1
M2H    1
M1H    1
M7Y    1
M6C    1
Name: Postcode, Length: 103, dtype: int64

<p>Since there are Postcodes with different Neighbourhoods, we will join them in a single cell.</p>

In [7]:
df["Neighbourhood"] = df.groupby("Postcode")["Neighbourhood"].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
...,...,...,...
254,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
261,M4Y,Downtown Toronto,Church and Wellesley
264,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
265,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [8]:
df.shape

(103, 3)

<h4><i>Geo-Spatial Data</i></h4>
<p>We will use the data from de csv file to have the longitude and latitude with the respective postal code</p>

In [9]:
geo_df=pd.read_csv("https://cocl.us/Geospatial_data")
geo_df.columns = ["Postcode","Latitude","Longitude"]
geo_df

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<p>Now that we have the data, we will merge it with the previous df (merging by postcode, of course).</p>

In [10]:
df = pd.merge(df,geo_df, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


<h4><i>Toronto Demographics Data</i></h4>
<p>We will use the data from Wikipedia to know the percentage of the population per Borough is Latin American.</p>

In [11]:
demo = requests.get("https://en.wikipedia.org/wiki/Demographics_of_Toronto").text
soup = BeautifulSoup(demo,"lxml")
#print(soup.prettify())

tables = soup.find_all("table",{"class":"wikitable sortable"})
#tables

In [12]:
dfs1 = pd.read_html(str(tables))
#dfs

In [13]:
torontoAndEastYork = dfs1[5]
northYork = dfs1[6]
scarborough = dfs1[7]
etobicokeAndYork = dfs1[8]

In [14]:
northYork[northYork["Ethnic Group #4"]=="Latin American"]

,Riding,Population,Ethnic Group #1,%,Ethnic Group #2,%.1,Ethnic Group #3,%.2,Ethnic Group #4,%.3,Ethnic Group #5,%.4,Ethnic Group #6,%.5
4,York Centre,103760,White,53.1,Filipino,16.5,Black,7.9,Latin American,5.1,NaN,NaN,NaN,NaN


In [15]:
torontoAndEastYork = torontoAndEastYork[torontoAndEastYork.eq("Latin American").any(1)]
northYork = northYork[northYork.eq("Latin American").any(1)]
scarborough = scarborough[scarborough.eq("Latin American").any(1)]
etobicokeAndYork = etobicokeAndYork[etobicokeAndYork.eq("Latin American").any(1)]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [16]:
def getIndexes(dfObj):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    value = "Latin American"
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [17]:
highLatinoPopulation = {'Riding':  ["East York",
                                    "North York",
                                    "York",
                                    "Etobicoke"
                                    ],
        'LatinoPercentage': [torontoAndEastYork.iloc[0,torontoAndEastYork.columns.get_loc(getIndexes(torontoAndEastYork)[0][1])+1],
                             max(northYork.iloc[0,northYork.columns.get_loc(getIndexes(northYork)[0][1])+1],
                             northYork.iloc[1,northYork.columns.get_loc(getIndexes(northYork)[1][1])+1]),
                             etobicokeAndYork.iloc[0,northYork.columns.get_loc(getIndexes(northYork)[0][1])+1],
                             etobicokeAndYork.iloc[0,northYork.columns.get_loc(getIndexes(northYork)[0][1])+1]
                            ]
        }
dfPopulationAUX = pd.DataFrame (highLatinoPopulation, columns = ['Riding','LatinoPercentage'])
dfPopulationAUX

,Riding,LatinoPercentage
0,East York,5.4
1,North York,9.5
2,York,8.5
3,Etobicoke,8.5


In [18]:
df[df.eq("York").any(1)]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
16,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
56,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
63,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262
64,M9N,York,Weston,43.706876,-79.518188


In [19]:
df[df["Borough"] == "Etobicoke"]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
17,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
70,M9P,Etobicoke,Westmount,43.696319,-79.532242
77,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
89,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
94,M9W,Etobicoke,Northwest,43.706748,-79.594054
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [20]:
import numpy as np
dfPopulation = pd.DataFrame({'Borough':df["Borough"].unique(),
                             'Population':np.zeros(len(df["Borough"].unique()))})

In [21]:
for x in range(0,len(dfPopulation["Borough"])):
    for y in range(0,len(dfPopulationAUX["Riding"])):
        if dfPopulation.iloc[x,0] == dfPopulationAUX.iloc[y,0]:
            dfPopulation.iloc[x,1] = dfPopulationAUX.iloc[y,1]
            print(dfPopulation.iloc[x,0])
            print(dfPopulationAUX.iloc[y,0])

North York
North York
Etobicoke
Etobicoke
East York
East York
York
York


In [22]:
dfPopulation.columns = ["Borough","LatinoPopulation"]
dfPopulation

,Borough,LatinoPopulation
0,North York,9.5
1,Downtown Toronto,0.0
2,Etobicoke,8.5
3,Scarborough,0.0
4,East York,5.4
5,York,8.5
6,East Toronto,0.0
7,West Toronto,0.0
8,Central Toronto,0.0
9,Mississauga,0.0


In [23]:
df['LatinoPopulation'] = np.zeros(len(df["Borough"]))

In [24]:
for x in range(0,len(df["Borough"])):
    for y in range(0,len(dfPopulation["Borough"])):
        if df.iloc[x,1] == dfPopulation.iloc[y,0]:
            df.iloc[x,5] = dfPopulation.iloc[y,1]

In [25]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0


<h4><i>Bicycle Parking Racks Data</i></h4>
<p>We will use the bicycle parking racks data to know how easy it would be for the restaurant to use delivery services.</p>

In [26]:
bicy_df = pd.read_csv("https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e930c873-18df-41ee-b2b2-bf293bc13e20?format=csv&projection=4326")
bicy_df.head()

,_id,ADDRESS_POINT_ID,ADDRESS_NUMBER,LINEAR_NAME_FULL,ADDRESS_FULL,POSTAL_CODE,MUNICIPALITY,CITY,CENTRELINE_ID,LO_NUM,...,CAPACITY,MULTIMODAL,SEASONAL,SHELTERED,SURFACE,STATUS,LOCATION,NOTES,MAP_CLASS,geometry
0,6026,51630,5,Bartonville Ave E,5 Bartonville Ave E,M6M 2B1,YORK,Toronto,20112838,5,...,8,No,No,No,asphalt,Delivered,NaN,No bolt down - in York District,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.49458..."
1,6027,310564,150,Borough Dr,150 Borough Dr,M1P 4N7,SCARBOROUGH,Toronto,107945,150,...,8,Yes,No,No,NaN,Installed,Scarborough Civic Centre,adjacent to civic centre & Square,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.25724..."
2,6028,367443,71,New Forest Sq,71 New Forest Sq,M1V 2Z6,SCARBOROUGH,Toronto,20050441,71,...,8,No,No,No,concrete,Installed,NaN,60m south of Steeles on W side of Kennedy (clo...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.30744..."
3,6029,379258,95,River Grove Dr,95 River Grove Dr,M1W 3T8,SCARBOROUGH,Toronto,20052191,95,...,8,No,No,No,concrete,Installed,NaN,S-W corner of Birchmount & Steeles (closest va...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.31738..."
4,6030,394585,24,Victoria Park Ave,24 Victoria Park Ave,M4E 3R9,SCARBOROUGH,Toronto,113274,24,...,8,No,No,No,NaN,Installed,NaN,One 8-bike rack on TTC property,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.28102..."


In [27]:
dropping = ['_id', 'ADDRESS_POINT_ID','ADDRESS_NUMBER','LINEAR_NAME_FULL','ADDRESS_FULL','CITY']
bicy_df.drop(dropping, axis=1, inplace=True)
bicy_df.head()

,POSTAL_CODE,MUNICIPALITY,CENTRELINE_ID,LO_NUM,LO_NUM_SUF,HI_NUM,HI_NUM_SUF,LINEAR_NAME_ID,WARD_NAME,X,...,CAPACITY,MULTIMODAL,SEASONAL,SHELTERED,SURFACE,STATUS,LOCATION,NOTES,MAP_CLASS,geometry
0,M6M 2B1,YORK,20112838,5,NaN,NaN,NaN,433,York South-Weston (505),305236.720,...,8,No,No,No,asphalt,Delivered,NaN,No bolt down - in York District,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.49458..."
1,M1P 4N7,SCARBOROUGH,107945,150,NaN,NaN,NaN,7397,Scarborough Centre (2121),324342.970,...,8,Yes,No,No,NaN,Installed,Scarborough Civic Centre,adjacent to civic centre & Square,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.25724..."
2,M1V 2Z6,SCARBOROUGH,20050441,71,NaN,NaN,NaN,9497,Scarborough-Agincourt (2222),320288.194,...,8,No,No,No,concrete,Installed,NaN,60m south of Steeles on W side of Kennedy (clo...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.30744..."
3,M1W 3T8,SCARBOROUGH,20052191,95,NaN,NaN,NaN,8803,Scarborough-Agincourt (2222),319488.988,...,8,No,No,No,concrete,Installed,NaN,S-W corner of Birchmount & Steeles (closest va...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.31738..."
4,M4E 3R9,SCARBOROUGH,113274,24,NaN,NaN,NaN,370,Beaches-East York (1919),322456.763,...,8,No,No,No,NaN,Installed,NaN,One 8-bike rack on TTC property,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.28102..."


<p> Since the Postal Code is in the long format, I will use only the first 3 characters of it.</p>

In [28]:
bicy_df['POSTAL_CODE'] = bicy_df['POSTAL_CODE'].astype(str).str[0:3]
bicy_df.head()

,POSTAL_CODE,MUNICIPALITY,CENTRELINE_ID,LO_NUM,LO_NUM_SUF,HI_NUM,HI_NUM_SUF,LINEAR_NAME_ID,WARD_NAME,X,...,CAPACITY,MULTIMODAL,SEASONAL,SHELTERED,SURFACE,STATUS,LOCATION,NOTES,MAP_CLASS,geometry
0,M6M,YORK,20112838,5,NaN,NaN,NaN,433,York South-Weston (505),305236.720,...,8,No,No,No,asphalt,Delivered,NaN,No bolt down - in York District,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.49458..."
1,M1P,SCARBOROUGH,107945,150,NaN,NaN,NaN,7397,Scarborough Centre (2121),324342.970,...,8,Yes,No,No,NaN,Installed,Scarborough Civic Centre,adjacent to civic centre & Square,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.25724..."
2,M1V,SCARBOROUGH,20050441,71,NaN,NaN,NaN,9497,Scarborough-Agincourt (2222),320288.194,...,8,No,No,No,concrete,Installed,NaN,60m south of Steeles on W side of Kennedy (clo...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.30744..."
3,M1W,SCARBOROUGH,20052191,95,NaN,NaN,NaN,8803,Scarborough-Agincourt (2222),319488.988,...,8,No,No,No,concrete,Installed,NaN,S-W corner of Birchmount & Steeles (closest va...,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.31738..."
4,M4E,SCARBOROUGH,113274,24,NaN,NaN,NaN,370,Beaches-East York (1919),322456.763,...,8,No,No,No,NaN,Installed,NaN,One 8-bike rack on TTC property,Multi-Bike Rack,"{u'type': u'Point', u'coordinates': (-79.28102..."


In order to make this data useful for our approach, I need the number of bicycles stands per postal code.

In [29]:
#bicy_count = bicy_df['POSTAL_CODE'].value_counts()
bicy_count = bicy_df['POSTAL_CODE'].value_counts().rename_axis('Postcode').reset_index(name='NumOfBikeStands')

In [30]:
bicy_count.head()

,Postcode,NumOfBikeStands
0,M5V,30
1,nan,29
2,M6G,16
3,M5H,16
4,M6H,15


In [31]:
bicy_count = bicy_count[bicy_count.Postcode != "nan"]
bicy_count.head()

,Postcode,NumOfBikeStands
0,M5V,30
2,M6G,16
3,M5H,16
4,M6H,15
5,M5G,14


In [32]:
df.Postcode
df['NumOfBikeStands'] = np.zeros(len(df.Postcode))
df.iloc[0,6]

0.0

In [33]:
for x in range(0,len(df.Postcode)):
    for y in range(0,len(bicy_count.Postcode)):
        if df.iloc[x,0] == bicy_count.iloc[y,0]:
            df.iloc[x,6] = bicy_count.iloc[y,1]

In [34]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0


In [35]:
bicy_count.iloc[0,1]

30

In [36]:
df3 = pd.read_csv("T120120190215054507.csv")
df3.head()

,Geographic code,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,Canada,NaN,T,35151728,15412443,14072079
1,A0A,A0A,Newfoundland and Labrador,NaN,46587,26155,19426
2,A0B,A0B,Newfoundland and Labrador,NaN,19792,13658,8792
3,A0C,A0C,Newfoundland and Labrador,NaN,12587,8010,5606
4,A0E,A0E,Newfoundland and Labrador,NaN,22294,12293,9603


In [37]:
df3 = df3[df3['Geographic code'].str.startswith('M')]
df3 = df3[['Geographic code', 'Population, 2016']]
df3.head()

,Geographic code,"Population, 2016"
895,M1B,66108
896,M1C,35626
897,M1E,46943
898,M1G,29690
899,M1H,24383


In [38]:
df3.columns = ["Postcode","Population"]
df3.head()

,Postcode,Population
895,M1B,66108
896,M1C,35626
897,M1E,46943
898,M1G,29690
899,M1H,24383


In [39]:
df = pd.merge(df,df3, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0,34615
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0,14443
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0,41078
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,10


In [40]:
df.shape

(102, 8)

<h4><i>Foursquare Data</i></h4>
<p>Now we will use the Foursquare Data to know the current offer of Latin American places
around the neighborhoods. After this step, we will begin clustering the different zones.</p>

In [41]:
CLIENT_ID = '05EQDIWX1LV1HHQFIVUGSUNEOBL5KHZLUII32WLMKE1FJWQ2' # your Foursquare ID
CLIENT_SECRET = 'JJKJMHS1Z3OJOP3RCPJDYXJGMOFRJWC2QHW0TRDODOPSMSYM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 05EQDIWX1LV1HHQFIVUGSUNEOBL5KHZLUII32WLMKE1FJWQ2
CLIENT_SECRET:JJKJMHS1Z3OJOP3RCPJDYXJGMOFRJWC2QHW0TRDODOPSMSYM


In [42]:
neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

search_query = 'Italian'
radius = 1000
print(search_query + ' .... OK!')

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.
Italian .... OK!


In [43]:
radius = 500 
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    neighborhood_latitude,
    neighborhood_longitude,
    VERSION,  
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=05EQDIWX1LV1HHQFIVUGSUNEOBL5KHZLUII32WLMKE1FJWQ2&client_secret=JJKJMHS1Z3OJOP3RCPJDYXJGMOFRJWC2QHW0TRDODOPSMSYM&ll=43.7532586,-79.3296565&v=20180604&radius=500&limit=30'

In [44]:
results = requests.get(url).json()

In [45]:
from pandas.io.json import json_normalize
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
queryDF = json_normalize(venues)
queryDF.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,venuePage.id
0,4e42684718a8627fce453c01,TTC stop #8380,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1584190285,False,Underhill Dr,At Cassandra N,43.752672,-79.326351,"[{'label': 'display', 'lat': 43.752672, 'lng':...",273,CA,Toronto,ON,Canada,"[Underhill Dr (At Cassandra N), Toronto ON, Ca...",NaN,NaN,NaN
1,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1584190285,False,Toronto,NaN,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]",NaN,NaN,NaN
2,55bbdfb9498e5996dd9d4038,Dollarama,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",v-1584190285,False,"1277 York Mills Rd,Parkwood Village",NaN,43.760341,-79.325519,"[{'label': 'display', 'lat': 43.760341, 'lng':...",855,CA,North York,ON,Canada,"[1277 York Mills Rd,Parkwood Village, North Yo...",M3A 1Z5,Parkwoods - Donalda,NaN
3,535fddb1498e03814e03968f,GTA Restoration | Emergency Water Damage Plumb...,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",v-1584190285,False,250 Yonge St,401 & DVP,43.753567,-79.351308,"[{'label': 'display', 'lat': 43.7535666482373,...",1741,CA,Toronto,ON,Canada,"[250 Yonge St (401 & DVP), Toronto ON M5B 2L7,...",M5B 2L7,NaN,84830578
4,4c4c83c646240f47898fe7f4,Pheasant Run Golf Course,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",v-1584190285,False,NaN,Warden & davis dr,43.758386,-79.337191,"[{'label': 'display', 'lat': 43.75838618241153...",832,CA,Sharon,ON,Canada,"[Sharon ON, Canada]",NaN,NaN,NaN


In [46]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in queryDF.columns if col.startswith('location.')] + ['id']
dataframe_filtered = queryDF.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,TTC stop #8380,Bus Stop,Underhill Dr,At Cassandra N,43.752672,-79.326351,"[{'label': 'display', 'lat': 43.752672, 'lng':...",273,CA,Toronto,ON,Canada,"[Underhill Dr (At Cassandra N), Toronto ON, Ca...",NaN,NaN,4e42684718a8627fce453c01
1,Brookbanks Park,Park,Toronto,NaN,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]",NaN,NaN,4e8d9dcdd5fbbbb6b3003c7b
2,Dollarama,Discount Store,"1277 York Mills Rd,Parkwood Village",NaN,43.760341,-79.325519,"[{'label': 'display', 'lat': 43.760341, 'lng':...",855,CA,North York,ON,Canada,"[1277 York Mills Rd,Parkwood Village, North Yo...",M3A 1Z5,Parkwoods - Donalda,55bbdfb9498e5996dd9d4038
3,GTA Restoration | Emergency Water Damage Plumb...,Construction & Landscaping,250 Yonge St,401 & DVP,43.753567,-79.351308,"[{'label': 'display', 'lat': 43.7535666482373,...",1741,CA,Toronto,ON,Canada,"[250 Yonge St (401 & DVP), Toronto ON M5B 2L7,...",M5B 2L7,NaN,535fddb1498e03814e03968f
4,Pheasant Run Golf Course,Golf Course,NaN,Warden & davis dr,43.758386,-79.337191,"[{'label': 'display', 'lat': 43.75838618241153...",832,CA,Sharon,ON,Canada,"[Sharon ON, Canada]",NaN,NaN,4c4c83c646240f47898fe7f4
5,Yorkmills Wellness & Spa,Spa,25 Lesmill Road Suite 200,NaN,43.756800,-79.325346,"[{'label': 'display', 'lat': 43.75680029671985...",524,CA,North York,ON,Canada,"[25 Lesmill Road Suite 200, North York ON, Can...",NaN,NaN,54ee51de498e7a6fbe4f00a7
6,Cassandra Public School,School,45 Cassandra Blvd,NaN,43.748291,-79.328889,"[{'label': 'display', 'lat': 43.7482913687093,...",556,CA,Toronto,ON,Canada,"[45 Cassandra Blvd, Toronto ON M3A 1S5, Canada]",M3A 1S5,NaN,4e039defd22d4cebf370894a
7,8th Don Mills Cub Pack,Other Great Outdoors,NaN,NaN,43.748055,-79.336710,"[{'label': 'display', 'lat': 43.74805549693462...",810,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,5057a346e4b09e4d63d45396
8,Three Valleys Public School,School,76 Three Valleys Dr.,at Laurentide Dr.,43.750595,-79.337341,"[{'label': 'display', 'lat': 43.75059473487960...",685,CA,Toronto,ON,Canada,"[76 Three Valleys Dr. (at Laurentide Dr.), Tor...",M3A 3B7,NaN,4b85b9e4f964a520586f31e3
9,All About Kids,Nursery School,NaN,NaN,43.755637,-79.338071,"[{'label': 'display', 'lat': 43.75563673030966...",726,CA,NaN,NaN,Canada,[Canada],NaN,NaN,4fd7bab1e4b045a9ebef0c75


In [47]:
# function that extracts the category of the venue
def get_category(row):
    try:
        cat_list = row['categories']
    except:
        cat_list = row['venue.categories']
        
    if len(cat_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [48]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
venus_df = getVenues(names = df['Postcode'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude']
                                  )

In [50]:
venues_df = venus_df

In [51]:
venues_df.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Now to see the current offer of general venues, we will count the number of them by Postcode.


In [52]:
venues_df.Postcode.value_counts()
venues_count = venues_df['Postcode'].value_counts().rename_axis('Postcode').reset_index(name='NumOfVenues')

In [53]:
venues_count

,Postcode,NumOfVenues
0,M5A,30
1,M4M,30
2,M6J,30
3,M5T,30
4,M5H,30
...,...,...
94,M1B,1
95,M5N,1
96,M9M,1
97,M9L,1


Now I will use this information in the original dataset.

In [54]:
df = pd.merge(df,venues_count, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0,34615,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0,14443,5
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0,41078,30
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048,15
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,10,30


In [55]:
df.shape

(99, 9)

Now, it would be interesting to know what is the most offered thing as venue in the area we are operating.

In [56]:
venues_df["Venue Category"].value_counts()

Coffee Shop         88
Café                69
Park                42
Restaurant          41
Pizza Place         37
                    ..
Sculpture Garden     1
Massage Studio       1
Church               1
Stationery Store     1
IT Services          1
Name: Venue Category, Length: 231, dtype: int64

In [57]:
# one hot encoding
onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

#rename the column 'Neighborhood' which represents a category name to 'Neighborhood Category' 
#this is to distinguish this column from the 'Neighborhood' column which we want to continue to use as the neighborhood name
onehot.rename(columns={'Neighborhood':'Neighborhood Category'}, inplace=True)

# add neighborhood column back to dataframe
onehot['Postcode'] = venues_df['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

In [58]:
onehot.shape

(1317, 232)

In [59]:
onehot

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,M8Z,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1313,M8Z,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1314,M8Z,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1315,M8Z,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
toronto_grouped = onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.shape

(99, 232)

Now we are going to merge this dataset with the previous one.

In [61]:
df = pd.merge(df,toronto_grouped, on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,Accessories Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0,34615,2,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0,14443,5,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0,41078,30,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048,15,0.066667,...,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.066667,0.000000
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,10,30,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.033333


In [62]:
df.shape

(99, 240)

<h4>Clustering</h4>

In [63]:
# set number of clusters
k = 5
drop = ['Postcode','Borough','Neighbourhood','Latitude','Longitude']
df_clustering = df.drop(drop, 1)
df_clustering.head()

,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,9.5,0.0,34615,2,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,9.5,0.0,14443,5,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.0,10.0,41078,30,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,9.5,0.0,21048,15,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.066667,0.000000
4,0.0,0.0,10,30,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.033333


In [64]:
from sklearn.preprocessing import StandardScaler

X = df_clustering.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[-0.45696872,  0.51584732, -1.02025352, ..., -0.13334888,
        -0.12394761, -0.26560301],
       [-0.45696872, -0.80854525, -0.74946237, ..., -0.13334888,
        -0.12394761, -0.26560301],
       [ 1.67698899,  0.94017556,  1.50713054, ..., -0.13334888,
        -0.12394761, -0.26560301],
       ...,
       [-0.45696872, -1.75614381,  0.24343851, ..., -0.13334888,
        -0.12394761,  5.23063455],
       [-0.45696872, -0.3584146 , -1.02025352, ..., -0.13334888,
        -0.12394761, -0.26560301],
       [-0.45696872, -0.63817054, -0.02735264, ...,  9.07702725,
        -0.12394761, -0.26560301]])

In [129]:
from sklearn.cluster import KMeans 
num_clusters = 4

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[1 1 0 2 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1
 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0
 1 1 0 0 0 1 1 0 1 1 3 1 1 1 1 0 1 1 0 0 1 0 1 1 0]


In [130]:
df["Labels"] = labels
df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,Accessories Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0,34615,2,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0,14443,5,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0,41078,30,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048,15,0.066667,...,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.066667,0.000000,2
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,10,30,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.033333,0


In [131]:
df.groupby('Labels').mean()

,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,Accessories Store,Airport,Airport Food Court,Airport Gate,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Labels,,,,,,,,,,,,,,,,,,,,,
0,43.668882,-79.396724,1.193548,4.387097,22268.483871,27.129032,0.000000,0.000000,0.000000,0.000000,...,0.005376,0.001075,0.001536,0.005376,0.000000,0.003226,0.001075,0.003557,0.000000,0.005837
1,43.722726,-79.391526,5.090909,0.696970,28613.363636,6.727273,0.000000,0.005051,0.000000,0.000000,...,0.000689,0.000000,0.001684,0.003608,0.000842,0.000000,0.000000,0.000000,0.003788,0.001789
2,43.718518,-79.464763,9.500000,0.000000,21048.000000,15.000000,0.066667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000
3,43.628947,-79.394420,0.000000,30.000000,49195.000000,17.000000,0.000000,0.058824,0.058824,0.058824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [132]:
df_venues2 = df.loc[:,"Accessories Store":"Yoga Studio"]

In [133]:
df_venues2["Postcode"] = df["Postcode"]

In [134]:
# get a list of columns
cols = list(df_venues2)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Postcode')))
cols

['Postcode',
 'Accessories Store',
 'Airport',
 'Airport Food Court',
 'Airport Gate',
 'Airport Lounge',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Workshop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Basketball Stadium',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bike Shop',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Breakfast Spot',
 'Brewery',
 'Bridal Shop',
 'Bubble Tea Shop',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Bus Station',
 'Business Service',
 'Butcher',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Church',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Arts Building',
 'College Gym',
 'College Rec Center',
 'College Stadiu

In [135]:
# use ix to reorder
df_venues2 = df_venues2.ix[:, cols]
df_venues2

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,M4A,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,M5A,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,M6A,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.066667,0.0,0.0,0.000000,0.000000,0.066667,0.000000
4,M7A,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.033333,0.000000,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M8X,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
95,M4Y,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.000000
96,M7Y,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.062500
97,M8Y,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [136]:
def return_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False).to_frame().reset_index()
    row_cutoff = row_categories_sorted.head(num_top_venues)
    row_cutoff.columns = ['Venue', 'Appears']
    return_array = row_cutoff['Venue'].where(row_cutoff['Appears'] != 0, other = np.NaN, axis = 0)

    return return_array.values

In [137]:
num = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_venuesSorted = pd.DataFrame(columns = columns)
df_venuesSorted['Postcode'] = df_venues2['Postcode']

for ind in np.arange(df_venues2.shape[0]):
    df_venuesSorted.iloc[ind, 1:] = return_common_venues(df_venues2.iloc[ind, :], num)

df_venuesSorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,Park,Food & Drink Shop,NaN,NaN,NaN
1,M4A,Intersection,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant
2,M5A,Coffee Shop,Park,Bakery,Café,Breakfast Spot
3,M6A,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop
4,M7A,Coffee Shop,Park,Creperie,Burger Joint,Sandwich Place


In [142]:
final_df = df.loc[:,"Postcode":"NumOfVenues"]
final_df = pd.merge(final_df,df_venuesSorted, on="Postcode")
final_df = pd.merge(final_df,df[['Postcode','Labels']], on="Postcode")
final_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,9.5,0.0,34615,2,Park,Food & Drink Shop,NaN,NaN,NaN,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,9.5,0.0,14443,5,Intersection,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,1
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,10.0,41078,30,Coffee Shop,Park,Bakery,Café,Breakfast Spot,0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048,15,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,2
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,10,30,Coffee Shop,Park,Creperie,Burger Joint,Sandwich Place,0


Now that we have clustered 4 different Postalcodes, we can see the characteristics of each of them, and choose a Neighborhood inside one of them to place the business.

<h4>Analysis 0</h4>

In [205]:
k0 = final_df[final_df.Labels == 0]
k0.iloc[0,0]

'M5A'

In [145]:
k0.mean()

Latitude               43.668882
Longitude             -79.396724
LatinoPopulation        1.193548
NumOfBikeStands         4.387097
Population          22268.483871
NumOfVenues            27.129032
Labels                  0.000000
dtype: float64

In [149]:
k0["1st Most Common Venue"].value_counts()

Café                9
Coffee Shop         9
Grocery Store       2
Clothing Store      1
Bar                 1
Park                1
Thai Restaurant     1
Dessert Shop        1
Greek Restaurant    1
Sandwich Place      1
Sushi Restaurant    1
Ramen Restaurant    1
Gift Shop           1
Gastropub           1
Name: 1st Most Common Venue, dtype: int64

In [150]:
k0["2nd Most Common Venue"].value_counts()

Coffee Shop               4
Clothing Store            2
Restaurant                2
Park                      2
Pizza Place               2
Café                      2
Seafood Restaurant        1
Farmers Market            1
Sandwich Place            1
Bakery                    1
Breakfast Spot            1
Sushi Restaurant          1
Bank                      1
Gastropub                 1
Japanese Restaurant       1
Juice Bar                 1
Italian Restaurant        1
Bar                       1
Mexican Restaurant        1
Movie Theater             1
Thrift / Vintage Store    1
Cocktail Bar              1
Asian Restaurant          1
Name: 2nd Most Common Venue, dtype: int64

In [151]:
k0["3rd Most Common Venue"].value_counts()

Italian Restaurant       5
Coffee Shop              5
Café                     4
Restaurant               2
Vietnamese Restaurant    2
Indian Restaurant        1
Juice Bar                1
Creperie                 1
Hotel                    1
Salon / Barbershop       1
Bakery                   1
Convenience Store        1
Farmers Market           1
Board Shop               1
Pet Store                1
Park                     1
Plaza                    1
Ice Cream Shop           1
Name: 3rd Most Common Venue, dtype: int64

<h4>Analysis 1</h4>

In [154]:
k1 = final_df[final_df.Labels == 1]

In [155]:
k1.mean()

Latitude               43.722726
Longitude             -79.391526
LatinoPopulation        5.090909
NumOfBikeStands         0.696970
Population          28613.363636
NumOfVenues             6.727273
Labels                  1.000000
dtype: float64

In [156]:
k1["1st Most Common Venue"].value_counts()

Park                          12
Pizza Place                    7
Grocery Store                  5
Coffee Shop                    3
Indian Restaurant              2
Discount Store                 2
Caribbean Restaurant           2
Bakery                         2
Golf Course                    1
Yoga Studio                    1
Bus Line                       1
American Restaurant            1
Health Food Store              1
Construction & Landscaping     1
Liquor Store                   1
Sandwich Place                 1
Baseball Field                 1
Convenience Store              1
Latin American Restaurant      1
Fast Food Restaurant           1
Middle Eastern Restaurant      1
Field                          1
Empanada Restaurant            1
Japanese Restaurant            1
Sporting Goods Shop            1
Spa                            1
Thai Restaurant                1
Jewelry Store                  1
College Stadium                1
Garden                         1
Gym       

In [157]:
k1["2nd Most Common Venue"].value_counts()

Convenience Store             4
Coffee Shop                   4
Bakery                        3
Playground                    3
Chinese Restaurant            3
Bus Line                      2
Pizza Place                   2
Pharmacy                      2
Middle Eastern Restaurant     1
Moving Target                 1
Neighborhood Category         1
Lounge                        1
Women's Store                 1
Café                          1
Sandwich Place                1
Mexican Restaurant            1
Gym                           1
Baseball Field                1
Bank                          1
Food Truck                    1
River                         1
Trail                         1
Department Store              1
Food & Drink Shop             1
Skating Rink                  1
Fast Food Restaurant          1
Home Service                  1
Construction & Landscaping    1
Athletics & Sports            1
Brewery                       1
Video Store                   1
Yoga Stu

In [159]:
k1["3rd Most Common Venue"].value_counts()

Coffee Shop                   3
Bank                          3
Sandwich Place                3
Pub                           2
Pizza Place                   2
Park                          2
Restaurant                    2
Trail                         2
Fried Chicken Joint           2
Pharmacy                      2
Pet Store                     1
Pool                          1
Seafood Restaurant            1
Convenience Store             1
Noodle House                  1
Metro Station                 1
Smoke Shop                    1
Liquor Store                  1
Baseball Field                1
Hotel                         1
Market                        1
Grocery Store                 1
Furniture / Home Store        1
Construction & Landscaping    1
Dance Studio                  1
Fast Food Restaurant          1
Swim School                   1
Electronics Store             1
Skating Rink                  1
Massage Studio                1
General Entertainment         1
Bakery  

<h4>Analysis 2</h4>

In [206]:
k2 = final_df[final_df.Labels == 2]
k2.iloc[0,0]

'M6A'

In [161]:
k2.mean()

Latitude               43.718518
Longitude             -79.464763
LatinoPopulation        9.500000
NumOfBikeStands         0.000000
Population          21048.000000
NumOfVenues            15.000000
Labels                  2.000000
dtype: float64

In [162]:
k2["1st Most Common Venue"].value_counts()

Clothing Store    1
Name: 1st Most Common Venue, dtype: int64

In [163]:
k2["2nd Most Common Venue"].value_counts()

Furniture / Home Store    1
Name: 2nd Most Common Venue, dtype: int64

In [164]:
k2["3rd Most Common Venue"].value_counts()

Accessories Store    1
Name: 3rd Most Common Venue, dtype: int64

<h4>Analysis 3</h4>

In [166]:
k3 = final_df[final_df.Labels == 3]

In [167]:
k3.mean()

Latitude               43.628947
Longitude             -79.394420
LatinoPopulation        0.000000
NumOfBikeStands        30.000000
Population          49195.000000
NumOfVenues            17.000000
Labels                  3.000000
dtype: float64

In [168]:
k3["1st Most Common Venue"].value_counts()

Airport Service    1
Name: 1st Most Common Venue, dtype: int64

In [169]:
k3["2nd Most Common Venue"].value_counts()

Airport Lounge    1
Name: 2nd Most Common Venue, dtype: int64

In [174]:
k3["3rd Most Common Venue"].value_counts()

Airport Terminal    1
Name: 3rd Most Common Venue, dtype: int64

In [196]:
data = {"k0": k0.mean().iloc[2:6],
        "k1": k1.mean().iloc[2:6],
        "k2": k2.mean().iloc[2:6],
        "k3": k3.mean().iloc[2:6],
       }

In [198]:
ks_df = pd.DataFrame(data) 

In [201]:
ks_df = ks_df.T

In [203]:
ks_df["Total Latino Population"] = (ks_df["LatinoPopulation"]/100)*ks_df["Population"]

In [204]:
ks_df

,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,Total Latino Population
k0,1.193548,4.387097,22268.483871,27.129032,265.785130
k1,5.090909,0.696970,28613.363636,6.727273,1456.680331
k2,9.500000,0.000000,21048.000000,15.000000,1999.560000
k3,0.000000,30.000000,49195.000000,17.000000,0.000000


In [207]:
k2

,Postcode,Borough,Neighbourhood,Latitude,Longitude,LatinoPopulation,NumOfBikeStands,Population,NumOfVenues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Labels
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,9.5,0.0,21048,15,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,2


<h3> Conclusion </h3>
We can choose between two different Postalcode, one is the k0 which is the M5A and the k2 which is the M6A.

The first one has a strong offer of venues and more bike stands, but the latino population is low. In the other hand, we have half the offer of venues and no bike stands but almost 2000 latinos living there. 

Comparably, the most common venue category of the first Postalcode is coffee places meanwhile in the only neighborhood of the second option the most common venue is Clothing Store (Coffee Shop is the third most common). It seems that the first option has more offer of restaurants than the second one.

In [210]:
from geopy.geocoders import Nominatim
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [217]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [218]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighbourhood'], final_df['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [208]:
ontario_geo = "Toronto2.geojson"
map_clusters.choropleth(geo_data=ontario_geo,
    data = final_df,
    columns=['Postal Code','Total Latino Population'],
    key_on='feature.properties.CFSAUID',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Number of schools per postal code')   

map_clusters


NameError: name 'map_clusters' is not defined